<a href="https://colab.research.google.com/github/m-edal/training_AutoML_EnvData/blob/main/tutorial_weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

prepare the Python environment

In [ ]:
!pip install obswx flaml matplotlib

# Tutorial of AutoML for weather data
---

Junjie Yu, Zhonghua Zheng, David Topping

The University of Mancherster, Manchester, UK

Language: Python

Packages: obswx, matplotlib, flaml, numpy, pandas, sklearn

- learn to process time series data
- learn to use the flaml library

Q: What is the time serials data?

A: Time series data is a sequence of data points collected at constant time intervals.
It is used in various fields, such as finance, weather forecasting, and signal processing.
Time series data can be analyzed to identify patterns, trends, and anomalies.

## 0 import the libraries
---

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from obswx import *
import numpy as np
from flaml import AutoML
import matplotlib.pyplot as plt
import numpy as np

## 1 download the data of the interested weather station
---
here we use `obswx` package to load the data.

Package homepage: https://envdes.github.io/obswx/index.html

### 1.1 download data from `obswx`

load the metadata.

Q: what is metadata?

A: The data about data.

In [ ]:
met = obswx(source='ISD')
# Load the metadata
met.get_meta(load=True).head() #use .head() to print 5 samples in the head of the data
# please check https://www.ncei.noaa.gov/products/land-based-station/integrated-surface-database
# to get more information about the table.

here we can select the weater **station** by combining USAF and WBAN

03334099999: Manchester airport.

Here to help select station: https://envdes.github.io/obswx/isd_map

In [ ]:
df = met.get_data(year=2023, station= "03334099999", isd_source="AWS")
df['TMP'] = df['TMP'].replace(',','.', regex=True).astype(float) # transform the temperature to float
df.index = pd.to_datetime(df['DATE']) # set the date as index

In [ ]:
df.head()

Q: What format is time data saved in?

A: The time data is saved in the format of datetime64[ns].

Q: How can I access the time data?

A: You can access the time data by using the index of the dataframe.

Q: Are there any types of time data that can be used?

A: timestamp, datetime (Python object), and datetime64[ns] (numpy object). Please refer to the following link for more information: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html

### 1.2 check the temperature of the raw data

TMP - Mean temperature (.1 Fahrenheit)

here we will not change the unit of temperature (Fahrenheit).

Q: How to present temperature? Which type of temperature is the standard?

A: Celsius (°C), Fahrenheit (°F), Kelvin (K). Kelvin is the standard unit in the International System of Units. Kelvin is an absolute temperature scale where 0 K is absolute zero, equivalent to -273.15°C. Celsius is commonly used in daily life, where Water freezes at 0°C and boils at 100°C (under standard atmospheric pressure). Fahrenheit is popular in the United States

Fahrenheit = Celsius * 5/9 + 32

Kelvin = Celsius + 273.15

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,3))
df['TMP'].plot(ax=ax) # check the data
ax.set_xlabel('Date')
ax.set_ylabel('Temperature (F)')
ax.set_title('Temperature of Manchester Airport')
fig.show()

we can see here are some **missing data** that are replaced by 9999.99

### 1.3 How to deal with the missing data?

1. interpolate
    - ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.interpolate.html
2. drop it
3. fill it with a constant value
    - mean
    - median
    - last observation
    - next observation
4. others (e.g. using a model to predict the missing value)

In [ ]:
# 1 find the missing value index
missing_value_index = df.query('TMP >= 9999').index
missing_value_index

In [ ]:
# 2 check the missing values
df.loc[missing_value_index]['TMP']

In [ ]:
# 3 fill the missing values using interpolation of linear method
df['T_filled'] = df['TMP'].apply(lambda x: x if x < 9999 else np.nan).interpolate(method='linear')

In [ ]:
# 4 print the values
print(df['T_filled'].loc[missing_value_index])

In [ ]:
# 5 plotting
fig, ax = plt.subplots(1,1, figsize=(10,3))
df['T_filled'].plot(ax=ax)
ax.set_xlabel('Date')
ax.set_ylabel('Temperature (F)')
ax.set_title('Temperature of Manchester Airport')
fig.show()

### 1.4 Try other station and fill the missing values with different methods and compare the different among them.


1. select station: https://envdes.github.io/obswx/isd_map

2. get 2023 **LEEK THORNCLIFFE** station data

```
    USAF: "033300"
    WBAN: "99999"
    Station Name: "LEEK THORNCLIFFE"
    Country: "UK"
    State: ""
    ICAO: ""
    Latitude: 53.133
    Longitude: -1.983
    Elevation: 299
    Begin: "20040706"
    End: "20240811"
```

3. fill the missing value using different methods (linear, nearest, polynomial (order2), last observation, and the next observation), and compare the filled value with mean, median.

In [ ]:
# try you code in here and the plot is like below

use the index to locate the data, and fill the missing value using with different methods

In [ ]:
#example code here:

## 2 train a time serial model using automl
---

### 2.1 download 12 years data

In [ ]:
df = pd.DataFrame()
for year in range(2010,2023): # 2012--2022 year
    df_ = met.get_data(year=year, station= "03334099999", isd_source="AWS")
    df = pd.concat([df, df_])

df['TMP'] = df['TMP'].replace(',','.', regex=True).astype(float) # transform the temperature to float
df.index = pd.to_datetime(df['DATE'])
df['T_filled'] = df['TMP'].apply(lambda x: x if x < 9999 else np.nan).interpolate(method='linear') # fill the missing values

In [ ]:
# plotting
fig, ax = plt.subplots(1,1, figsize=(10,3))
df['T_filled'].plot(ax=ax)
ax.set_xlabel('Date')
ax.set_ylabel('Temperature (F)')
ax.set_title('Temperature of Manchester Airport')
fig.show()

### 2.3 resample the data to monthly data

In [ ]:
df_m = df['T_filled'].resample('M').mean().reset_index()
df_m

### 2.4 Using `flaml` automl package to model the time serial data

In [ ]:
# 1 initialize an instance of automl
automl = AutoML()

# 2 use the data excluding last 12 data for training
X_train = df_m.iloc[0:len(df_m)-12]

# 3 train the model
automl.fit(
    dataframe=X_train,  # dataframe with the column of timestamp
    label="T_filled",  # value column name
    period=12,  # time horizon to forecast, e.g., 12 months
    task="ts_forecast",
    time_budget=15,  # time budget in seconds
    log_file_name="ts_forecast.log",
    eval_method="holdout",
    estimator_list=[
        "lgbm",
        "xgboost",
        "extra_tree",
    ],
)

Notes: 
holdout is used to split the data into training and testing data, e.g., the last 10% data is used for testing and the rest is used for training. Use `split_ratio` to change the ratio.

In [ ]:
# 4 use the last 12 data for testing
df_m_test = df_m.iloc[len(df_m)-12:]['DATE']

In [ ]:
# 5 precdtion using the model
prediction = automl.predict(df_m_test)

### 2.5 Ploting

In [ ]:
# 6 plot the result
fig, ax = plt.subplots(1,1, figsize=(10,3))
ax.plot(df_m.iloc[len(df_m)-12:]['T_filled'], label='True')
ax.plot(prediction, label='Prediction')
date = df_m.iloc[len(df_m)-12:]['DATE'].dt.strftime('%Y-%m')
ax.set_xticklabels(date)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Temperature (F)')
ax.set_title('Temperature of Manchester Airport')
fig.show()

**Additional Questions:**

Q: How does time horizon to forecast (the parameter period) will affect the model performance?

In [ ]:
# try to revise the period to 1 month
# check and plot the precdtions



### 2.6 calculate the error metrics

\begin{equation}
    \text{MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2,
\end{equation}

\begin{equation}
    \text{RMSE} = \sqrt{\text{MSE}} = \sqrt{\frac{1}{n} \sum_{i=1}^n (y_i - \hat{y}_i)^2},
\end{equation}

\begin{equation}
    \text{MAE} = \frac{1}{n} \sum_{i=1}^n |y_i - \hat{y}_i|,
\end{equation}

\begin{equation}
    \text{R2} = 1 - \frac{\sum_{i=1}^n (y_i - \hat{y}_i)^2}{\sum_{i=1}^n (y_i - \bar{y_m})^2},
\end{equation}

where $y_i$ and $y{\hat{}}_i$ are the observations and model predictions, respectively; n is the number of samples; $y_m$ is the mean of the observations.

In [ ]:
# example: calculate the MSE, RMSE, MAE and R2 using the equations


## 3 Daily temperature prediction
---

In [ ]:
df_D = df['T_filled'].resample('D').mean().reset_index()

In [ ]:
automl = AutoML()
X_train = df_D.iloc[0:len(df_D)-30]
X_train['T_filled'] = X_train['T_filled'].fillna(method='ffill')
automl.fit(
    dataframe=X_train,  # dataframe with the column of timestamp
    label="T_filled",  # value column name
    period=1,  # time horizon to forecast
    task="ts_forecast",
    time_budget=60,  # time budget in seconds
    log_file_name="ts_forecast.log",
    eval_method="holdout",
    estimator_list=[
        "lgbm"
    ],
)

In [ ]:
test_D = df_D.iloc[len(df_D)-300:]['DATE']
prediction = automl.predict(test_D)

In [ ]:
# ploting
fig, ax = plt.subplots(1,1, figsize=(10,3))
ax.plot(df_D.iloc[len(df_D)-300:]['T_filled'], label='True')
ax.plot(prediction.loc[len(df_D)-300:len(df_D)-30], label='Prediction 1')
ax.plot(prediction.loc[len(df_D)-30:], label='Prediction 2')
date = df_D.iloc[len(df_D)-300:]['DATE'].dt.strftime('%Y-%m-%d')
ax.set_xticklabels(date)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Temperature (F)')
ax.set_title('Temperature of Manchester Airport')
fig.show()

the performances of the model is not good.

Q: Why?

1. Model are not well trained.
2. Task are not suitable for the ML modeling
3. Use more variables

Try to modify some paramters of automl

e.g., increasing the `time_budget`. Please try to increase the `time_budget` to 120 second and see what happen.


Q: **Think about the difference in prediction 1 and prediction 2**

## 4 Use more variables to help modeling
---

here we try to use more variables to help modeling

- TMP: temperature
- DEW: dew point

Q: [**what is dew point?**](https://en.wikipedia.org/wiki/Dew_point)

A: The dew point of a given body of air is the temperature to which it must be cooled to become saturated with water vapor. It is very related to the humdity.

In [ ]:
# 1 define a function to get more variables
def isd_data_process(var):
  df[var] = df[var].replace(',','.', regex=True).astype(float) # transform the temperature to float
  df[f'{var}_filled'] = df[var].apply(lambda x: x if x < 9999 else np.nan).interpolate(method='linear')

# 2 select TMP, DEW, SLP
for var in ['TMP', 'DEW']:
  isd_data_process(var)

In [ ]:
df_md = df[['TMP_filled','DEW_filled']].resample('D').mean().reset_index().fillna(method='ffill') # fill the na using ffill
automl = AutoML()
X_train = df_md.iloc[0:len(df_md)-30]
automl.fit(
    dataframe=X_train,  # dataframe with the column of timestamp
    label="TMP_filled",  # value column name
    period=1,  # time horizon to forecast
    task="ts_forecast",
    time_budget=60,  # time budget in seconds
    log_file_name="ts_forecast.log",
    eval_method="holdout",
    estimator_list=[
        "lgbm"
    ],
)

In [ ]:
test_md = df_md.iloc[len(df_md)-300:][['DATE',	'DEW_filled']]
prediction = automl.predict(test_md)

In [ ]:
# ploting
fig, ax = plt.subplots(1,1, figsize=(10,3))
ax.plot(df_md.iloc[len(df_md)-300:]['TMP_filled'], label='True')
ax.plot(prediction.loc[len(df_md)-300:len(df_md)-30], label='Prediction 1')
ax.plot(prediction.loc[len(df_md)-30:], label='Prediction 2')
date = df_md.iloc[len(df_md)-300:]['DATE'].dt.strftime('%Y-%m-%d')
ax.set_xticklabels(date)
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Temperature (F)')
ax.set_title('Temperature of Manchester Airport')
fig.show()

### Task:
---

1. Find the XIAOSHAN airport station (China, Hangzhou) data from 2010 to 2023, and fill the missing value if have.

2. Train two time-series models

    1. montly temperature prediction (1-month period)
    
    2. daily temperature prediction (3-day periods)

    3. try to use different `eval_method` methods.

3. Predict the 2024/01--2024/07 monthly temperature and compare the result with the true value.

4. Plot and calculate the MAE.

*optional: try to train model use Kelvin and Celsius